## Section Headings Curation and sentences spliter of Chilean Policies

In this notebook we will go through the list of documents i each country to create a dictionary of the first 7 characters of the document code associated to a country.

In [1]:
from pathlib import Path
import boto3, json, operator, os, re, string
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

### Connection to the AWS S3 bucket
To effectively run this cell you need Omdena's credentials. Please keep them local and do not sync them in GitHub repos nor cloud drives. Before doing anything with this json file, please think of security!!

In [2]:
json_folder = Path("C:/Users/user/Google Drive/Els_meus_documents/projectes/CompetitiveIntelligence/WRI/Notebooks/credentials/")
# json_folder = Path("C:/Users/jordi/Google Drive/Els_meus_documents/projectes/CompetitiveIntelligence/WRI/Notebooks/credentials/")
filename = "Omdena_key_S3.json"
file = json_folder / filename

with open(file, 'r') as f:
    cred = json.load(f) 

for key in cred:
    KEY = key
    SECRET = cred[key]

s3 = boto3.resource(
    service_name = 's3',
    region_name = 'us-east-2',
    aws_access_key_id = KEY,
    aws_secret_access_key = SECRET
)

### Functions

In [3]:
policy_dict = {'Credit' : 'Credit', 'Direct' : 'Direct payment', 'Fine' : 'Fine', 'General' : 'Unknown', 
               'Guarantee' : 'Credit', 'Supplies' : 'Supplies', 'Tax' : 'Tax deduction', 
               'Technical' : 'Technical assistance', 'Unknown' : 'Unknown', 'Other' : 'Unknown', 'Nan' : 'Unknown' }
policy_counter = {"Credit" : 0, "Direct payment" : 0, "Fine" : 0, "Supplies" : 0, "Tax deduction" : 0, "Technical assistance" : 0}
incentive_counter = {"Incentive" : 0, "not_Incentive" : 0}

incentive_dict = {'Incentive' : 'Incentive', 'Disincentive' : 'Incentive', 'Unknown' : 'not_Incentive', 'Nan' : 'not_Incentive'}

In [22]:
def clean_df(df, reference_column_to_drop_na):
    df.dropna(subset = [reference_column_to_drop_na], inplace = True)
    df.dropna(axis=1, how='all', inplace = True)
    df.replace(np.nan, 'Unknown', regex=True, inplace = True)
    return df

def process_new_labels(filename, reference_column_to_drop_na, policy):
    df_temp = pd.concat(pd.read_excel(filename, engine='openpyxl', sheet_name = None, skiprows=[0]), ignore_index = True)
    df_temp = clean_df(df_temp, reference_column_to_drop_na)
    df_temp.insert(0, "Document", df_temp.apply(lambda row: row.Sentence_Id.split("_")[0], axis = 1))
    df_temp.loc[df_temp['Is_policy'] == 0, 'Is_policy'] = "Unknown"
    df_temp.loc[df_temp['Is_policy'] == 1, 'Is_policy'] = policy
    if "Other_instrument"in df_temp.columns:
        df_temp['Is_policy'] = np.where(df_temp['Is_policy'] == "Unknown", df_temp['Other_instrument'], df_temp['Is_policy'])
    df_temp.loc[df_temp['Is_incentive'] == 0, 'Is_incentive'] = "not_incentive"
    df_temp.loc[df_temp['Is_incentive'] == 1, 'Is_incentive'] = "Incentive"
    return df_temp

def label_cleaning(dictionaryionary, label):
    flag = 1
    
    for key in dictionaryionary:
#         print(key, "----", label)
        if key in label:
            return dictionaryionary[key]
            flag = 0
            break
    if flag == 1:
#         print(label)
        return label

def merge_excel_to_list_new(filename):
    flag = False
    
    for policy in policy_dict:
        if policy in filename:
            policy_instrument = policy
            flag = True
    if flag:
#         print(filename)
        df = process_new_labels(filename, "Is_incentive", policy_instrument)
        List = df[["Document", "Sentence_Id", "Sentence", "Is_policy", "Is_incentive"]]
    
    return List


### Pipeline to process files from S3 bucket
By executing this cell you will go through all policies in El Salvador and process section headings that will be saved in a dictionary. This should be merged with the notebook that builds up the final json files out of plain txt files.

In [16]:
in_folder = {"Chile/full/" : "Chile", "El Salvador/" : "El Salvador"}
out_folder = "JSON/"

# bag_of_words = {}
# sentences = []
# sentences_dict = {}
json_file = {}
for path in in_folder:
    for obj in s3.Bucket('wri-latin-talent').objects.all().filter(Prefix= path):
        if path in obj.key and obj.key.replace(path, "") != "":# and filename in obj.key   # Un comment the previous string to run the code just in one sample document.
            key = obj.key.replace(path,"")
            json_file[key[0:7]] = in_folder[path]
           
       

In [49]:
# for item in json_file:
#     if "El Salvador" in json_file[item]:
#         print(item)

raters = ["Rater1", "Rater2", "Rater3"]
i = 0
rater = "Rater2"
base_path = Path("C:/Users/user/Google Drive/Els_meus_documents/projectes/CompetitiveIntelligence/WRI/Notebooks/Data/Final_input_data/{}/".format(rater))
data_path = base_path.glob('**/')
for path in data_path:
#         results_list = []    
    path_in_str = str(path)
#         print(path_in_str)
    if rater in path_in_str:
        dictionary = {}
#         print(path_in_str)
        for file_obj in Path(path_in_str).glob('*.xlsx'):
            file = str(file_obj)
            print(file[0:-5])
            df = merge_excel_to_list_new(file)
            df['Country'] = [json_file[x] if x in json_file else "El Salvador" for x in df['Document']]
            df.to_csv(file[0:-5] + ".csv", index=False)
    

C:\Users\user\Google Drive\Els_meus_documents\projectes\CompetitiveIntelligence\WRI\Notebooks\Data\Final_input_data\Rater2\Unique_sentence_IDs_Credit
C:\Users\user\Google Drive\Els_meus_documents\projectes\CompetitiveIntelligence\WRI\Notebooks\Data\Final_input_data\Rater2\Unique_sentence_IDs_Direct_payment
C:\Users\user\Google Drive\Els_meus_documents\projectes\CompetitiveIntelligence\WRI\Notebooks\Data\Final_input_data\Rater2\Unique_sentence_IDs_Fine
C:\Users\user\Google Drive\Els_meus_documents\projectes\CompetitiveIntelligence\WRI\Notebooks\Data\Final_input_data\Rater2\Unique_sentence_IDs_Supplies
C:\Users\user\Google Drive\Els_meus_documents\projectes\CompetitiveIntelligence\WRI\Notebooks\Data\Final_input_data\Rater2\Unique_sentence_IDs_Tax_deduction
C:\Users\user\Google Drive\Els_meus_documents\projectes\CompetitiveIntelligence\WRI\Notebooks\Data\Final_input_data\Rater2\Unique_sentence_IDs_Technical_assistance


In [50]:
df.sort_values(by = ["Country"], ascending = False).head(50)

,Document,Sentence_Id,Sentence,Is_policy,Is_incentive,Country
32,7bd959f,7bd959f_195,Participar en la promocion y ejecucion de los ...,Unknown,not_incentive,El Salvador
2,eb2af8d,eb2af8d_43,e) velar por el buen funcionamiento y operacio...,Unknown,not_incentive,El Salvador
96,731dbf0,731dbf0_34,Los aspectos tecnicos y administrativos relaci...,Unknown,not_incentive,El Salvador
101,43f0373,43f0373_145,Fomentar la aplicacion de las normas tecnicas ...,Technical,Incentive,El Salvador
41,4c71364,4c71364_43,Supervisor: Profesional de la arquitectura o d...,Unknown,not_incentive,El Salvador
63,1f15406,1f15406_99,autorizacion las condicionantes tecnicas de ma...,Unknown,not_incentive,El Salvador
62,9d724a3,9d724a3_64,Vigilar el cumplimiento de las especificacione...,Unknown,not_incentive,El Salvador
123,43f0373,43f0373_150,Verificar que las entidades responsables de el...,Unknown,not_incentive,El Salvador
22,8436b4f,8436b4f_19,Se crearan infraestructuras y recursos que fac...,Supplies,Incentive,El Salvador
12,6ed3f17,6ed3f17_177,Los estudios tecnicos requeridos correran por ...,Unknown,not_incentive,El Salvador


In [ ]:
# Loading back the sentences dictionary
out_folder = Path("C:/Users/user/Google Drive/Els_meus_documents/projectes/CompetitiveIntelligence/WRI/Notebooks/Data/Processed/")
filename = "Chile_V2_210109.json"
file = out_folder / filename
with open(file, 'r') as fp:
    json_file = json.load(fp)

In [ ]:
print(len(json_file))
# for k in sorted(sentences_dict):
#     print(k, ":", sentences_dict[k])

#### Short summary

In [ ]:
print("After preprocessing there are {} different headings in El Salvador policies".format(len(bag_of_words)))
print("{} documents have been processed".format(i))
print("There are {} lines of text as sentences".format(len(sentences)))

#### Dictionary items sorted by occurrence

In [ ]:
dict( sorted(bag_of_words.items(), key=operator.itemgetter(1),reverse=True))

#### Dictionary items sorted by heading text

In [ ]:
for k in sorted(bag_of_words):
    print(k, ":", bag_of_words[k])

In [ ]:
sentence_length = {}
for filename in json_file:
    for section in json_file[filename]:
        for sentence_id in json_file[filename][section]["sentences"]:
            length = len(json_file[filename][section]["sentences"][sentence_id]["text"])
            if length in sentence_length:
                sentence_length[length] += 1
            else:
                sentence_length[length] = 1
            if length == 13:
                print(sentence_id, "--", json_file[filename][section]["sentences"][sentence_id])


In [ ]:
sentence_length_list = []
for key, value in sentence_length.items():
    sentence_length_list.append([round(key), round(value)])

In [ ]:
sorted_length = sorted(sentence_length_list,key=lambda x: x[0])
df = pd.DataFrame(sorted_length, columns = ['Sentence_length','Frequency'])

In [ ]:
round(df.describe())

In [ ]:
df[len(df)-20:len(df)]

In [ ]:
x_max = df["Sentence_length"].max()
x_max = 65
x_sep = 1
df[0:x_max].plot.bar(x='Sentence_length', y='Frequency', rot=45, figsize = (18, 10), xticks=range(0, x_max, x_sep), xlim = (0, x_max))

In [ ]:
sorted_arr = np.asarray(sorted_length)
labels, ys = zip(*sorted_arr)
xs = np.arange(len(labels)) 
width = 1

plt.bar(xs, ys, width, align='center')

plt.xticks(xs, labels) #Replace default x-ticks with xs, then replace xs with labels
plt.yticks(ys)

# plt.savefig('netscore.png')

#### Saving sentences as csv

In [ ]:
print(sentences[0:2])

In [ ]:
# path = Path("C:/Users/user/Google Drive/Els_meus_documents/projectes/CompetitiveIntelligence/WRI/Notebooks/Data/")
path = Path("C:/Users/jordi/Google Drive/Els_meus_documents/projectes/CompetitiveIntelligence/WRI/Notebooks/Data/")
filename = "sentences.npy"
file = path / filename
np_sentences = np.array(sentences)
with open(file, 'wb') as f:
    np.save(f, np_sentences)

### Pipeline to process one file from HD folder
This is a pipeline to process a test file in a local folder.

In [ ]:
path = "C:/Users/jordi/Google Drive/Els_meus_documents/projectes/CompetitiveIntelligence/WRI/Documents_de_mostra/Chile/"
# path = "C:/Users/user/Google Drive/Els_meus_documents/projectes/CompetitiveIntelligence/WRI/Documents_de_mostra/Chile/"
files = os.listdir(path)
print(files[0])

In [ ]:
# path = "C:/Users/user/Google Drive/Els_meus_documents/projectes/CompetitiveIntelligence/WRI/Documents_de_mostra/Chile/"
path = "C:/Users/jordi/Google Drive/Els_meus_documents/projectes/CompetitiveIntelligence/WRI/Documents_de_mostra/Chile/"
data_folder = Path(path)
filename = "00d91bcac2667290e3fb42452f5241be28b031c2"


files = os.listdir(path)
files = ["002c53058e85d383b057fa4cc25a6eb8e7d401e3"]

bag_of_words = {}
json_file = {}

i = 0
for filename in files:
#     if i == 0:
    file_ = data_folder / filename
    with open(file_, 'r', encoding = 'utf-8') as file:
        lines = file.readlines()
#         print("\n", filename, "\n")
        json_file[filename] = {}
        line_counter = 0
        heading_flag = True
        heading_content = False
        has_section = False
        json_file[filename]["HEADING"] = {"tags" : [], "sentences" : {}}
        for line in lines:
            line = clean_whitespace(line)
            # Processing document heading
            if heading_flag:
                if "Tipo Norma" in line:
                    heading_content = True
                if heading_content:
                    line = full_cleaning(line)
                    if line != None:
                        if ":" in line:
                            line_counter += 1
                            sentence_id = filename[0:7] + '_' + str(line_counter)
                            json_file[filename]["HEADING"]["sentences"][sentence_id] = {"text" : line, "labels" : []}
                            json_file[filename]["HEADING"]["sentences"][sentence_id]["text"] = line
                        else:
                            json_file[filename]["HEADING"]["sentences"][sentence_id]["text"] = json_file[filename]["HEADING"]["sentences"][sentence_id]["text"] + " " + line
#                 print("**", line)
                heading_flag, heading_content = end_of_heading(line, heading_flag, heading_content)

            # Breaking when document signatures are found    
            elif end_of_document(line):
#                 print(line)
                break
            # Getting section headings
            elif (uppercase_ratio(line) == 1 and len(line) > 10 and line_counter > 0) or is_section(line):
#                 print("line--", line)
                line = remove_accents(line)
                line = clean_bugs(line)
                line = clean_sentence(line)
                if line == None:
                    continue
                else:
                    has_section = True
                    section = merge_concepts(line)
#                     print("**", section)
                    json_file[filename][section] = {"tags" : [], "sentences" : {}}
#                     bag_of_words = add_to_dict(section, bag_of_words, duplicates_dict)
                    if section == "VISTO" and len(split_into_sentences(line, ":")) > 1:
                        visto = split_into_sentences(line, ":")
                        for sentence in split_into_sentences(visto[1], ";"):
                            line_counter += 1
                            sentence_id = filename[0:7] + '_' + str(line_counter)
                            json_file[filename][section]["sentences"][sentence_id] = {"text" : sentence, "labels" : []}

            elif has_section:
                line = full_cleaning(line)
                if line == None:
                    continue                    
                else:
                    line = change_decimal_points(line)
                    for sentence in split_into_sentences(line, "."):
                        line_counter += 1
                        sentence_id = filename[0:7] + '_' + str(line_counter)
                        json_file[filename][section]["sentences"][sentence_id] = {"text" : sentence, "labels" : []}
        i += 1
    #     data = file.read().replace('\n', '')

In [ ]:
json_file

#### Dictionary items sorted by occurrence

In [ ]:
dict( sorted(bag_of_words.items(), key=operator.itemgetter(1),reverse=True))

#### Dictionary items sorted by heading text

In [ ]:
for k in sorted(bag_of_words):
    print(k, ":", bag_of_words[k])